In [ ]:
import matplotlib.pyplot as plt
import sys
import logging
import os
from cellpose import models, train
from sklearn.model_selection import train_test_split
import napari
from magicgui import magicgui
import numpy as np
from tifffile import imwrite

src_path = os.path.dirname(os.getcwd())
sys.path.append(src_path)
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('napari').setLevel(logging.WARNING)
logging.getLogger('in_n_out').setLevel(logging.WARNING)
logging.getLogger('numcodecs').setLevel(logging.WARNING)
logging.getLogger('numba').setLevel(logging.WARNING)

from src import Receipt, load_data
from src.steps import download_data, return_data

In [ ]:
receipt = Receipt(
    analysis_name = 'cellpose_segmentation',
    nas_location = r'Users\Jack\_ImageDatasets\IntronDiffusion\08072025_TriptolideTimeSeries_JF001\JF001_15min_5uM_3',
    local_location = None,
    data_loader = 'pycromanager_data_loader'
    )

In [ ]:
receipt = download_data(receipt, 'download_data').process()

In [ ]:
# Parameters
mask_name = 'handseg_MCPGFP_2'
channel_of_interest = 1
model_name = 'DEFAULT_MODEL'
do_3d = False

In [ ]:
data = load_data(receipt)
images = data['images'][:,:,channel_of_interest,:,:,:]

# Start viewer
viewer = napari.Viewer()

# Add image and labels
viewer.add_image(images, name="image")

# Run Napari
napari.run()



In [ ]:
# After closing viewer, retrieve labels array
final_labels = viewer.layers['Labels'].data
imwrite(os.path.join(receipt['dirs']['masks_dir'], f'{mask_name}.tif'), final_labels)

print("Labels shape:", final_labels.shape)
print("Unique labels:", np.unique(final_labels))

In [ ]:
return_data(receipt, 'return_data').process()

In [ ]:
# load data
data = load_data(receipt)
images = data['images'][:,:,channel_of_interest,:,:,:]
labels = data[mask_name]

# image_list = [np.expand_dims(np.asarray(images[p, t]), axis=-1) for p in range(images.shape[0]) for t in range(images.shape[1])]
if do_3d:
    image_list = [np.asarray(images[p, t]) for p in range(images.shape[0]) for t in range(images.shape[1])]
    label_list = [np.asarray(labels[p, t]) for p in range(labels.shape[0]) for t in range(labels.shape[1])]
else:
    image_list = [np.max(np.asarray(images[p, t]),axis=0) for p in range(images.shape[0]) for t in range(images.shape[1])]
    label_list = [np.max(np.asarray(labels[p, t]),axis=0) for p in range(labels.shape[0]) for t in range(labels.shape[1])]

image_list = [im[np.newaxis, ...] for im in image_list]
# label_list = [lbl[np.newaxis, ...] for lbl in label_list]

# split data
training_images, testing_images, training_labels, testing_labels = train_test_split(
    image_list, label_list, test_size=0.2, random_state=42
)

# assert np.all(im.shape == lbl.shape for im, lbl in zip(training_images, training_labels))
# assert np.all(im.shape == lbl.shape for im, lbl in zip(testing_images, testing_labels))

# train new model
model = models.CellposeModel(gpu=True, model_type='livecell')

path = train.train_seg(model.net,
                    train_data=training_images, 
                    train_labels=training_labels,
                    test_data=testing_images, 
                    test_labels=testing_labels,
                    channel_axis=0,
                    weight_decay=0.1, 
                    learning_rate=1e-5,
                    n_epochs=100, 
                    model_name=model_name, 
                    min_train_masks=1
                    )



In [ ]:
# train_losses = model.net.train_losses
# test_losses = model.net.test_losses

# # display losses
# plt.figure(figsize=(10, 5))
# plt.plot(train_losses, label='Training Loss')
# plt.plot(test_losses, label='Testing Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.title('Training and Testing Losses')
# plt.show()